# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [99]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
cities_df = pd.read_csv('../Resources/Cities.csv')
cities_df = cities_df.drop(columns='Unnamed: 0')

cities_df.head()


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Aasiaat,68.71,-52.87,31.64,95,100,8.03
1,Abalak,58.13,68.59,36.14,89,14,7.38
2,Abu Samrah,35.30,37.18,57.15,69,0,9.01
3,Acapulco,16.86,-99.89,86.00,66,20,12.75
4,Adiake,5.29,-3.30,86.00,84,40,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [118]:
gmaps.configure(api_key=g_key)
humidity_locations = cities_df[['Latitude', 'Longitude']]
humidity = cities_df['Humidity']
layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '5px',
    'margin': '0 auto'
}
heatmap_fig = gmaps.figure(center = (19.4326,-99.1332), zoom_level=3, layout=layout)
heat_layer = gmaps.heatmap_layer(
    humidity_locations,
    weights = humidity,
    dissipating = False,
    max_intensity = 100,
    point_radius = 1
)
heatmap_fig.add_layer(heat_layer)
heatmap_fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto', padding='5px', width='80…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [119]:
max_temp = 77
min_temp = 57.2
wind_speed_lim = 6
cloudiness = 40
vacation_cities_df = cities_df[
    (cities_df['Temperature'] >= min_temp) & 
    (cities_df['Temperature'] <= max_temp) &
    (cities_df['Wind Speed'] <= wind_speed_lim) &
    (cities_df['Cloudiness'] <= cloudiness)
]

vacation_cities_df = vacation_cities_df.reset_index()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
#Do not run this unless you want to restart hotel data frame
hotel_df = vacation_cities_df
hotel_df['Hotel Name'] = ''
hotel_df['Country'] = ''
print(hotel_df.shape)
hotel_df.head()

(43, 10)


,index,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Country
0,8,Agadir,30.42,-9.60,64.40,88,0,3.36,,
1,10,Airai,-8.93,125.41,57.22,72,27,3.83,,
2,25,Ambovombe,-25.17,46.08,67.08,79,0,1.23,,
3,29,Anshun,26.25,105.93,61.38,60,37,1.16,,
4,57,Bethel,41.37,-73.41,68.22,30,1,5.82,,


In [108]:

params = {
    'types': 'lodging',
    'radius': 5000,
    'key': g_key
}

g_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for (idx, row) in hotel_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    location = f'{lat}, {lon}'
    params['location'] = location
    response = requests.get(g_base_url, params=params).json()
    try: 
        hotel_df.loc[idx, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[idx, 'Country'] = response['results'][0]['plus_code']['compound_code'].split(',')[-1].strip()
    except:
        print(f'Could not get a hotel for the city {row["City"]}')
    

Could not get a hotel for the city Ikalamavony
Could not get a hotel for the city Otse
Could not get a hotel for the city Tsiroanomandidy


In [124]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']

In [121]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [122]:
locations.shape

(40, 2)

In [135]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_names = list(hotel_df['Hotel Name'])
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_names, display_info_box=True)
heatmap_fig.add_layer(markers)
heatmap_fig.add_layer(marker_layer)
heatmap_fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto', padding='5px', width='80…